A regular expression (regex) is a sequence of characters that describes a search pattern. We can use regular expressions to search for and extract data.

In practice, we say that strings match a regular expression if the pattern exists anywhere within those strings (as substrings). The simplest example of a regular expression is an ordinary sequence of characters that we specify. We say that any string containing that sequence of characters, adjacent and in the same exact order, matches the regular expression.

In [1]:
strings = ["data science", "big data", "metadata"]
regex = "data"  # common in all the above three elements

There are a number of special characters we can use with regular expressions to change the way a pattern is interpreted. In Python, we use the re module to work with regular expressions. The module's documentation provides a list of these special characters.

For instance, we use the special character "." to indicate that any character can be put in its place. Here are a few examples of how you might use this placeholder:

In [2]:
strings = ["bat", "robotics", "megabyte"]
regex = "..t" #regular expression that is three characters long and matches every string in strings

We can use the caret symbol ("^") to match the beginning of a string, and the dollar sign ("$") to match the end of a string.
"^a" will match all strings that start with "a".
"a$" will match all strings that end with "a".

In [3]:
strings = ["better not put too much", "butter in the", "batter"]
regex = "^b.tter"

Dataset - Reddit
Columns
    Title -- The title of the post
    Score -- The number of upvotes the post received
    Time -- When the post was posted
    Gold -- How much Reddit Gold users gave the post
    NumComs -- The number of comments the post received


In [4]:
import csv
f = open("askreddit_2015.csv", 'r')
csvreader = csv.reader(f)
posts_with_header = list(csvreader)
posts = posts_with_header[1:]
for post in posts[:10]:
    print(post)

['What\'s your internet "white whale", something you\'ve been searching for years to find with no luck?', '11510', '1433213314.0', '1', '26195']
["What's your favorite video that is 10 seconds or less?", '8656', '1434205517.0', '4', '8479']
['What are some interesting tests you can take to find out about yourself?', '8480', '1443409636.0', '1', '4055']
["PhD's of Reddit. What is a dumbed down summary of your thesis?", '7927', '1440188623.0', '0', '13201']
['What is cool to be good at, yet uncool to be REALLY good at?', '7711', '1440082910.0', '0', '20325']
['[Serious] Redditors currently in a relationship, besides dinner and a movie, what are your favorite activities for date night?', '7598', '1439993280.0', '2', '5389']
["Parents of Reddit, what's something that your kid has done that you pretended to be angry about but secretly impressed or amused you?", '7553', '1439161809.0', '0', '11520']
['What is a good subreddit to binge read the All Time Top Posts of?', '7498', '1438822288.0',

One useful function the module provides is re.search.
With re.search(regex, string), we can check whether string is a match for regex. If it is, the expression will return a match object. If it isn't, it will return None. For now, we won't worry about returning the actual matches - we'll just compare the result to None to see whether we have a match or not.


In [6]:
import re

of_reddit_count = 0
for row in posts:
    if re.search("of Reddit", row[0]) is not None:
        of_reddit_count += 1
print('of_reddit_count is ',of_reddit_count)        

('of_reddit_count is ', 76)


If you look at the data set closely, you may notice that some posts use "of Reddit", and others use "of reddit". While both versions have the same format, the capitalization of "Reddit" is different. We can account for this inconsistency with square brackets. We use square brackets in a regex to indicate that any character within them can fill the space.

For example, the regex "[bcr]at" would match the substrings "bat", "cat", and "rat", but nothing else. We indicate that the first character in the regex can be either "b", "c" or "r".

In [7]:
import re

of_reddit_count_old = 0
for row in posts:
    if re.search("of Reddit", row[0]) is not None:
        of_reddit_count_old += 1
of_reddit_count = 0
for row in posts:
    if re.search("of [Rr]eddit",row[0]) is not None:
        of_reddit_count+=1

Our data set contains a lot of posts that use the [Serious] tag. AskReddit members use this tag to indicate that they're not looking for humorous responses, and that their question should be taken seriously. We'd like to search through our data set to see how many posts have this tag, but the regex "[Serious]" doesn't do what we need. Since square brackets serve a special function within regular expressions, "[Serious]" will match any string that contains "S", "e", "r", etc.

To deal with this sort of problem, we need to escape special characters. In regular expressions, escaping a character means indicating that you don't want the character to do anything special, and that the interpreter should treat it just like any other character. We use the backslash ("\") to escape characters in a regex.

Suppose we wanted to match all of the strings that end with a period. If we used ".$", it would match all strings that contain any character, because "." has that special meaning. Instead, we need to escape the "." with a backslash, so our regex becomes "\.$".

In [8]:
import re

serious_count = 0
for row in posts:
    if re.search("\[Serious\]",row[0]) is not None:
        serious_count+=1
print('serious_count is',serious_count)        
        

('serious_count is', 69)


In [9]:
import re 
serious_count=0
for row in posts:
    if re.search("\[[Ss]erious\]",row[0]) is not None: #[Ss] is NOT ignored hence no /
        serious_count +=1
print('serious_count is',serious_count)        

('serious_count is', 77)


In our data set, some users have tagged their posts with "(Serious)" or "(serious)" and "[serious] or [Serious]", including the parentheses. Therefore, we should account for both square brackets and parentheses.

In [10]:
serious_count=0
for row in posts:
    if re.search("[\[\(][Ss]erious[\]\)]",row[0]) is not None:
        serious_count+=1
print('serious_count is',serious_count)        

('serious_count is', 80)


We should consider a post serious only if the tag occurs at the beginning or end of the title. To match titles with the tag at the beginning, we can use the "^" character in a regex. To match titles with the tag at the end, we can use "$". These characters produce two different regular expressions, and we'd like to identify all titles that match either of them.

To combine regular expressions, we use the "|" character. For example, "cat|dog" would match "catfish" and "hotdog", because both of these strings match either "cat" or "dog". Similarly, we can combine our regular expressions for the serious tags with the "|" operator to match all titles that either begin or end with the tag.

In [11]:

serious_start_count = 0
serious_end_count = 0
serious_count_final = 0
for row in posts:
    if re.search("^[\[\(][Ss]erious[\]\)]",row[0]) is not None:
        serious_start_count+=1
    if re.search("[\[\(][Ss]erious[\]\)]$",row[0]) is not None:
        serious_end_count+=1
    if re.search("^[\[\(][Ss]erious[\]\)]|[\[\(][Ss]erious[\]\)]$",row[0]) is not None: #Note no space b/w |
        serious_count_final +=1
print('serious_start_count is',serious_start_count)  
print('serious_end_count is',serious_end_count)  
print('serious_count_final is',serious_count_final)  

('serious_start_count is', 69)
('serious_end_count is', 11)
('serious_count_final is', 80)


The re module provides a sub() function that takes the following parameters (in order):

    pattern: The regex to match
    repl: The string that should replace the substring matches
    string: The string containing the pattern we want to search

If we were to call re.sub("yo", "hello", "yo world"), the function will replace the "yo" in "yo world" with "hello", producing the result "hello world". If it doesn't find a pattern, the re.sub() function simply returns the original string.

Let's use re.sub() to convert all serious tags to the format "[Serious]".

In [12]:
posts_new = []
for row in posts:
    row[0]=re.sub("[\[\(][Ss]erious[\]\)]","[Serious]",row[0])
    posts_new.append(row)

In [19]:
strings=['War of 1812', 'There are 5280 feet to a mile', 'Happy New Year 2016!']
year_strings = []
for row in strings:
    if re.search("[1-2][0-9][0-9][0-9]",row) is not None: # only year b/w 1000-2999
        year_strings.append(row)
print('year_strings is',year_strings)        

('year_strings is', ['War of 1812', 'Happy New Year 2016!'])


 we used the regex "[1-2][0-9][0-9][0-9]", which looks a bit repetitive. Luckily, there's a better way to do it!

We can use curly brackets ("{" and "}") to indicate that a pattern should repeat. To match any four-digit number, for example, we could repeat the pattern "[0-9]" four times by writing "[0-9]{4}".

In [21]:
year_strings = []
for row in strings:
    if re.search("[1-2][0-9]{3}",row):# reapeting the patter[0-9] three times ie,[0-9]{3} ==[0-9][0-9][0-9]
        year_strings.append(row)
print('year_strings is',year_strings)          

('year_strings is', ['War of 1812', 'Happy New Year 2016!'])


Finally, let's extract years from a string. The re module contains a findall() function that returns a list of substrings matching the regex. re.findall("[a-z]", "abc123") would return ["a", "b", "c"], because those are the substrings that match the regex.

In [22]:
years_string='2015 was a good year, but 2016 will be better!'
years=re.findall("[1-2][0-9]{3}",years_string)
print("years is",years)

('years is', ['2015', '2016'])
